In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import matplotlib.pyplot  as plt
import seaborn as sns
import networkx as nx
from datetime import datetime
from itertools import product
from sqlalchemy.orm import sessionmaker
from datetime import datetime, timedelta
import networkx as nx
import scipy as sp
from scipy.special import comb
import random
import copy
import seaborn as sns
import matplotlib.dates as mdates
from shapely import wkb
import binascii
from pyproj import Transformer
from collections import defaultdict
plt.rcParams["figure.figsize"] = [20, 6]
plt.rcParams["figure.dpi"] = 300
plt.rcParams["xtick.labelsize"] = 15 
plt.rcParams["ytick.labelsize"] = 15 
plt.rcParams["axes.labelsize"] = 25
plt.rcParams["legend.fontsize"] = 20  

In [ ]:
# database credentials
db_usr, db_pwd = '', '' # your database user name and password

# database login
host, port, db = 'nc-health-data-prod.cluster-ccsgl7rk4urn.eu-central-1.rds.amazonaws.com', 5432, 'master'
engine = create_engine('postgresql://'+db_usr+':'+db_pwd+'@'+host+':'+str(port)+'/'+db)
Session = sessionmaker(bind=engine)
session = Session()
conn = engine.connect()

In [ ]:
def consolidate_list(lst):
    mapping = {}
    
    for item in lst:
        # Use a tuple of the sorted first two elements as a key for the dictionary
        key = tuple(sorted(item[:2]))
        if key in mapping:
            mapping[key].append(item[2])
        else:
            mapping[key] = [item[2]]
    # Convert the dictionary back to the desired list format
    result = [(k[0], k[1], mapping[k]) for k in mapping.keys()]

    return result
def sort_time(weight_list):
    data = weight_list
    # Extract the timestamps and create a DataFrame
    df = pd.DataFrame(data, columns=["Latitude", "Longitude", "Timestamp"])

    # Sort by Timestamp
    df = df.sort_values(by="Timestamp")

    # Group and select the first timestamp of each group
    df['group'] = (df['Timestamp'].diff() > pd.Timedelta(minutes=60)).cumsum()
    df = df.groupby('group').first().reset_index(drop=True)
    tuples = [tuple(row) for row in df.itertuples(index=False)]

    return tuples

In [ ]:

# Creating the DataFrame with the updated match information, German city names, and times
# Creating the DataFrame with the updated match information, German city names, and times
data_group_a = {
    'City': ['München', 'Köln', 'Stuttgart', 'Köln', 'Frankfurt am Main', 'Stuttgart'],
    'Date': ['2024-06-14', '2024-06-15', '2024-06-19', '2024-06-19', '2024-06-23', '2024-06-23'],
    'Time': ['21:00', '15:00', '18:00', '21:00', '21:00', '21:00'],
    'Population': [1471508, 1085664, 634830, 1085664, 753056, 634830],
    'Match': [
        'Germany 5 - 1 Scotland', 
        'Hungary 1 - 3 Switzerland', 
        'Germany 2 - 0 Hungary', 
        'Scotland 1 - 1 Switzerland', 
        'Switzerland 1 - 1 Germany', 
        'Scotland 0 - 1 Hungary'
    ]
}
df_group_a = pd.DataFrame(data_group_a)

data_group_b = {
    'City': ['Berlin', 'Dortmund', 'Hamburg', 'Gelsenkirchen', 'Leipzig', 'Düsseldorf'],
    'Date': ['2024-06-15', '2024-06-15', '2024-06-19', '2024-06-20', '2024-06-24', '2024-06-24'],
    'Time': ['18:00', '21:00', '15:00', '21:00', '21:00', '21:00'],
    'Population': [3769000, 588462, 1841179, 260000, 597493, 619294],
    'Match': [
        'Spain 3 - 0 Croatia', 
        'Italy 2 - 1 Albania', 
        'Croatia 2 - 2 Albania', 
        'Spain 1 - 0 Italy', 
        'Croatia 1 - 1 Italy', 
        'Albania 0 - 1 Spain'
    ]
}
df_group_b = pd.DataFrame(data_group_b)

data_group_c = {
    'City': ['Stuttgart', 'Gelsenkirchen', 'München', 'Frankfurt am Main', 'Köln', 'München'],
    'Date': ['2024-06-16', '2024-06-16', '2024-06-20', '2024-06-20', '2024-06-25', '2024-06-25'],
    'Time': ['18:00', '21:00', '15:00', '18:00', '21:00', '21:00'],
    'Population': [634830, 260000, 1472000, 753056, 1085664, 1472000],
    'Match': [
        'Slovenia 1 - 1 Denmark', 
        'Serbia 0 - 1 England', 
        'Slovenia 1 - 1 Serbia', 
        'Denmark 1 - 1 England', 
        'England 0 - 0 Slovenia', 
        'Denmark 0 - 0 Serbia'
    ]
}

df_group_c= pd.DataFrame(data_group_c)

data_group_d = {
    'City': ['Hamburg', 'Düsseldorf', 'Berlin', 'Leipzig', 'Berlin', 'Dortmund'],
    'Date': ['2024-06-16', '2024-06-17', '2024-06-21', '2024-06-21', '2024-06-25', '2024-06-25'],
    'Time': ['15:00', '21:00', '18:00', '21:00', '18:00', '18:00'],
    'Population': [1841179, 619294, 3769000, 597493, 3769000, 588462],
    'Match': [
        'Poland 1 - 2 Netherlands', 
        'Austria 0 - 1 France', 
        'Poland 1 - 3 Austria', 
        'Netherlands 0 - 0 France', 
        'Netherlands 2 - 3 Austria', 
        'France 1 - 1 Poland'
    ]
}
df_group_d = pd.DataFrame(data_group_d)


data_group_e = {
    'City': ['München', 'Frankfurt am Main', 'Düsseldorf', 'Köln', 'Frankfurt am Main', 'Stuttgart'],
    'Date': ['2024-06-17', '2024-06-17', '2024-06-21', '2024-06-22', '2024-06-26', '2024-06-26'],
    'Time': ['15:00', '18:00', '15:00', '21:00', '18:00', '18:00'],
    'Population': [1488202, 753056, 617280, 1080394, 753056, 635911],
    'Match': [
        'Romania 3 - 0 Ukraine', 
        'Belgium 0 - 1 Slovakia', 
        'Slovakia 1 - 2 Ukraine', 
        'Belgium 2 - 0 Romania', 
        'Slovakia 1 - 1 Romania', 
        'Ukraine 0 - 0 Belgium'
    ]
}

df_group_e = pd.DataFrame(data_group_e)


# Creating the DataFrame with the match information, cities, dates, times, and populations
data_group_f = {
    'City': ['Dortmund', 'Leipzig', 'Hamburg', 'Dortmund', 'Gelsenkirchen', 'Hamburg'],
    'Date': ['2024-06-18', '2024-06-18', '2024-06-22', '2024-06-22', '2024-06-26', '2024-06-26'],
    'Time': ['18:00', '21:00', '15:00', '18:00', '21:00', '21:00'],
    'Population': [588462, 597493, 1841179, 588462, 260000, 1841179],
    'Match': [
        'Turkey 3 - 1 Georgia', 
        'Portugal 2 - 1 Czechia', 
        'Georgia 1 - 1 Czechia', 
        'Turkey 0 - 3 Portugal', 
        'Georgia 2 - 0 Portugal', 
        'Czechia 1 - 2 Turkey'
    ]
}
df_group_f = pd.DataFrame(data_group_f)




data_eurocup_16th = {
    'City': ['Dortmund', 'Berlin', 'Gelsenkirchen', 'Köln', 'Düsseldorf', 'Frankfurt am Main', 'München', 'Leipzig'],
    'Date': ['2024-06-29', '2024-06-29', '2024-06-30', '2024-06-30', '2024-07-01', '2024-07-01', '2024-07-02', '2024-07-02'],
    'Time': ['21:00', '18:00', '18:00', '21:00', '18:00', '21:00', '18:00', '21:00'],
    'Population': [588462, 3644826, 260000, 1085664, 617280, 736414, 1471508, 597493],
    'Match': [
        'Germany 2 - 0 Denmark',
        'Switzerland 2 - 0 Italy',
        'England 2 - 1 Slovakia ',
        'Spain 4 - 1 Georgia',
        'France 1 - 0 Belgium',
        'Portugal 0 - 0 Slovenia',
        'Romania 0 - 3 Netherlands',
        'Austria 1 - 2 Turkey'
    ]
}

df_eurocup_16th = pd.DataFrame(data_eurocup_16th)

# Creating the dataframe for the semifinals
data_semifinals = {
    'City': ['München', 'Dortmund'],
    'Date': ['2024-07-09', '2024-07-10'],
    'Time': ['21:00', '21:00'],
    'Population': [1471508, 588462],
    'Match': [
        'Spain 2 - 1 France',
        'Netherlands 1 - 2 England'
    ]
}

df_semifinals = pd.DataFrame(data_semifinals)

# Creating the dataframe for the quarterfinals
data_quarterfinals = {
    'City': ['Stuttgart', 'Hamburg', 'Berlin', 'Düsseldorf'],
    'Date': ['2024-07-05', '2024-07-05', '2024-07-06', '2024-07-06'],
    'Time': ['18:00', '21:00', '18:00', '21:00'],
    'Population': [634830, 1841179, 3644826, 617280],
    'Match': [
        'Spain 2 - 1 Germany',
        'Portugal 0 - 0 France ',
        'Netherlands 2 - 1 Turkey',
        'England 1 - 1 Switzerland'
    ]
}

df_quarterfinals = pd.DataFrame(data_quarterfinals)

data_final = {
    'City': ['Berlin'],
    'Date': ['2024-07-14'],
    'Time': ['21:00'],
    'Population': [3644826],
    'Match': [
        'Spain 2 - 1 England'
    ]
}

df_final = pd.DataFrame(data_final)



# Creating the DataFrame with the match information, cities, dates, times, and populations


df_group_a['Group'] = 'Group A'
df_group_b['Group'] = 'Group B'
df_group_c['Group'] = 'Group C'
df_group_d['Group'] = 'Group D'
df_group_e['Group'] = 'Group E'
df_group_f['Group'] = 'Group F'

# Combine the DataFrames
combined_df = pd.concat([df_group_a, df_group_b, df_group_c, df_group_d, df_group_e, df_group_f, df_eurocup_16th, df_quarterfinals, df_semifinals, df_final ], ignore_index=True)
print(combined_df)

## (Fig 5) Heatmaps per city

In [ ]:
slots = 24
start_date = datetime.strptime('2024-05-06', '%Y-%m-%d')
end_date = datetime.strptime('2024-07-30', '%Y-%m-%d')
city='Berlin'

In [ ]:
import re
# Define FIFA country codes
fifa_country_codes = {
    'Germany': 'GER', 'Scotland': 'SCO', 'Hungary': 'HUN', 'Switzerland': 'SUI', 'Spain': 'ESP',
    'Croatia': 'CRO', 'Italy': 'ITA', 'Albania': 'ALB', 'Slovenia': 'SVN', 'Denmark': 'DEN',
    'Serbia': 'SRB', 'Poland': 'POL', 'Netherlands': 'NED', 'Austria': 'AUT', 'France': 'FRA',
    'Romania': 'ROU', 'Ukraine': 'UKR', 'Belgium': 'BEL', 'Slovakia': 'SVK', 'Turkey': 'TUR',
    'Georgia': 'GEO', 'Portugal': 'POR', 'Czechia': 'CZE'
}

# Function to replace country names with FIFA codes
def map_fifa_codes(match_str):
    # Extract teams using regex
    match = re.match(r'^(.*) (\d+) - (\d+) (.*)$', match_str)
    if match:
        team1, score1, score2, team2 = match.groups()
        # Map country names to FIFA codes
        team1_code = fifa_country_codes.get(team1.strip(), team1.strip())
        team2_code = fifa_country_codes.get(team2.strip(), team2.strip())
        return f"{team1_code} {score1} - {score2} {team2_code}"
    return match_str

In [ ]:
# Event details from combined_df
event_details = combined_df[['City', 'Date', 'Time', 'Match']]

# Convert Date column to datetime
event_details['Date'] = pd.to_datetime(event_details['Date'])

# Extract event times and dates
event_times = {
    (row['City'], datetime.strptime(row['Date'].strftime('%Y-%m-%d') + ' ' + row['Time'], '%Y-%m-%d %H:%M')): row['Match']
    for index, row in event_details.iterrows()
    if start_date <= row['Date'] <= end_date
}
delta = end_date - start_date
days_list = [(start_date + timedelta(days=i)).strftime('%Y-%m-%d') for i in range(delta.days + 1)]
print(days_list)
date_range = [start_date + timedelta(days=x) for x in range((end_date - start_date).days + 1)]

number_of_days = len(date_range)


In [ ]:
from collections import defaultdict



df_total = pd.DataFrame()
data_dict = {}
for day in days_list:
    query =f"""
    SELECT *
    FROM (
        SELECT cnv.*, t2p.*
        FROM covid_network_sdkv6_tl5_10m cnv 
        JOIN tile2plz t2p ON cnv.tile_id = t2p.tile_id
        WHERE "day" = '{day}'
    ) t1
    JOIN covid_plz_populate cpp ON t1.plz = cpp.id_plz5 
    WHERE city = '{city}'
    """


    didlist=[]
    pairlist=[]

    df_day= pd.DataFrame(pd.read_sql_query(query, conn))
    transformer = Transformer.from_crs("epsg:3857", "epsg:4326")

    # iterate over contact events
    for dids, sources, in_stad, point, time in zip(df_day.dids, df_day.sources, df_day.dist_stads, df_day.geopoint, df_day.stime):
        ngps = 1 if (in_stad==True) else 0
        for i, si in zip(dids, sources[1:-1].split(',')):
            for j, sj in zip(dids, sources[1:-1].split(',')):
                if i < j and [si, sj].count('GPS') >= ngps:
                    # collect dids in did list
                    didlist.append(i)
                    didlist.append(j)
                    bytes_object = binascii.unhexlify(point)
                    geometry = wkb.loads(bytes_object)
                    # Transformer from EPSG:3857 to EPSG:4326

                    # Assuming that `geometry` is a Point
                    lon, lat = transformer.transform(geometry.x, geometry.y)
                    # collect pairs of dids in pair list
                    pairlist.append((i, j, (lon, lat, time)))

    # get unique dids and did pairs
    print(f"Checkpoint 1 at: {datetime.now()}")
    didlist, pairlist = list(set(didlist)), list(set(pairlist))
    print(len(didlist))

    G = nx.Graph()
    G.add_nodes_from(didlist)
    print(f"Checkpoint 2 at: {datetime.now()}")

    pairlist_multiple = consolidate_list(pairlist)
    pairlist_multiple = [(node1, node2, sort_time(another_list)) for node1, node2, another_list in pairlist_multiple]
    G.add_weighted_edges_from(pairlist_multiple)
    print(len(G.edges()))
    print(len(G.nodes()))
    edges = list(G.edges(data=True))

    for edge in edges:
        node1, node2, attr = edge
        for weight in attr['weight']:
            matrix = np.zeros((1, slots), dtype=int)
            gps_point = weight[:2]
            timestamp = weight[2]
            if timestamp.tzinfo is not None:
                timestamp = timestamp.tz_localize(None)
            hour = timestamp.hour
            minute = timestamp.minute
            interval_index = (hour * 60 + minute) // 60
            matrix[0][interval_index] = 1
            key = (node1, node2, day)
            if key in data_dict:
                data_dict[key]['vector'] += matrix.flatten()
                data_dict[key]['gps_points'].append(gps_point)
            else:
                data_dict[key] = {
                    'did1': node1,
                    'did2': node2,
                    'vector': matrix.flatten(),
                    'gps_points': [gps_point],
                    'day': day
                }


In [ ]:
data_dict

In [ ]:
date_range = [start_date + timedelta(days=x) for x in range((end_date - start_date).days + 1)]
number_of_days = len(date_range)
organized_data = defaultdict(lambda: np.zeros((number_of_days, slots)))


for (did1, did2, date_str), info in data_dict.items():
    date = datetime.strptime(date_str, '%Y-%m-%d')
    index = (date - start_date).days
    if 0 <= index < number_of_days:
        organized_data[(did1, did2)][index, :] = info['vector']
    else:
        print(f"Index {index} is out of bounds for ({did1}, {did2}) with date {date_str}")

df_rows = []

for (did1, did2), vectors in organized_data.items():
    # Create a full vector matrix initialized with zeros
    full_vector_matrix = np.zeros((number_of_days, slots))

    # Directly assign the vectors to the corresponding dates
    full_vector_matrix[:vectors.shape[0], :vectors.shape[1]] = vectors
    # Sum the slots per day
    daily_sums = full_vector_matrix.sum(axis=1)


    # Append to the DataFrame rows
    df_rows.append({'pair': f'{did1}-{did2}', 'vector': full_vector_matrix, 'daily_sums': daily_sums})






In [ ]:
df = pd.DataFrame(df_rows)
df['flattened_vector'] = df['vector'].apply(lambda x: x.reshape(-1))

number_of_days = len(df['flattened_vector'].iloc[0]) // slots
heatmap_data = np.zeros((number_of_days, slots))
for _, row in df.iterrows():
    matrix = np.array(row['vector']).reshape(number_of_days, slots)
    heatmap_data += matrix
    
heatmap_data_df = pd.DataFrame(heatmap_data)


## To replot 

In [ ]:
city='Berlin'
date_range = [start_date + timedelta(days=x) for x in range((end_date - start_date).days + 1)]
number_of_days = len(date_range)
combined_df['Match'] = combined_df['Match'].apply(map_fifa_codes)
# Event details from combined_df
event_details = combined_df[['City', 'Date', 'Time', 'Match']]

# Convert Date column to datetime
event_details['Date'] = pd.to_datetime(event_details['Date'])

# Extract event times and dates
event_times = {
    (row['City'], datetime.strptime(row['Date'].strftime('%Y-%m-%d') + ' ' + row['Time'], '%Y-%m-%d %H:%M')): row['Match']
    for index, row in event_details.iterrows()
    if start_date <= row['Date'] <= end_date
}

heatmap_data_df=pd.read_csv(f'data/heatmaps_16m_10m/heatmap_data_{city}_tl7_10m.csv')

heatmap_data_df = heatmap_data_df.iloc[5:]
heatmap_data = np.flipud(heatmap_data_df.T)


In [ ]:
import matplotlib.colors as mcolors

sns.set_context("notebook")
sns.set_style("white")

# Assuming heatmap_data, start_date, number_of_days, slots, city, and event_times are already defined

vmax_auto = np.max(heatmap_data)
print(vmax_auto)
fig, ax = plt.subplots(figsize=(40, 15))

# Create the heatmap without the color bar
cax = sns.heatmap(heatmap_data, cmap='YlGnBu', vmin=0, vmax=180, ax=ax, linewidths=.5)
cbar = cax.collections[0].colorbar
cbar.ax.tick_params(labelsize=50)

# Generate all dates within the range
dates = [start_date + timedelta(days=i) for i in range(number_of_days)]

# Identify all Sundays
sundays = [date for date in dates if date.weekday() == 6]

# Select every other Sunday
sundays_every_two_weeks = sundays[::2]

# Positions of the selected Sundays in the dates list
sunday_positions = [dates.index(sunday) for sunday in sundays_every_two_weeks]

# Set x-ticks at the positions of the selected Sundays
ax.set_xticks(sunday_positions)

# Set x-tick labels as the formatted dates
xtick_labels = [sunday.strftime('%m/%d') for sunday in sundays_every_two_weeks]
ax.set_xticklabels(xtick_labels, ha='right', fontsize=60)

# Set y-ticks every 4 hours
ax.set_yticks([i + 3.5 for i in range(0, slots, 4)])  # Ticks at 0, 4, 8, 12, 16, 20

# Set y-tick labels corresponding to every 4 hours
ax.set_yticklabels(reversed(range(0, slots, 4)), fontsize=60, rotation=0)

# Move the y-axis label slightly to the right
ax.set_ylabel("hour of the day", fontsize=75, labelpad=30)  # Adjust 'labelpad' as needed

ax.set_title(f"{city}", fontsize=80)

# Define a threshold for deciding if the text color should be orange
threshold = 40  # Adjust this value based on your data

# Plot events
for (event_city, event_time), event_name in event_times.items():
    if event_city == city:
        event_day_index = (event_time.date() - start_date.date()).days
        event_hour_index = slots - 1 - event_time.hour  # Adjusted for 24 slots

        match_parts = event_name.split(" ")
        team1 = match_parts[0][:3].upper()
        score1 = match_parts[1]
        score2 = match_parts[3]
        team2 = match_parts[4][:3].upper()

        team_abbreviation = f"{team1}-{team2}"
        
        # Determine if the value in heatmap_data is below the threshold
        cell_value = heatmap_data[event_hour_index, event_day_index]
        if cell_value < threshold:
            text_color = 'black'
            font_weight = 'bold'
        else:
            text_color = 'black'
            font_weight = 'bold'
        
        ax.text(event_day_index + 0.5, event_hour_index + 0.5, team_abbreviation,
                ha='center', va='center', color=text_color, fontsize=30, rotation=90)

plt.tight_layout()
ax.set_xlabel("day", fontsize=75, labelpad=30)

# Corrected file path handling
#output_file = rf"C:nc-euro24\plots\heatmap_{city}_tl7_10m.svg"

#plt.savefig(output_file, format="svg", dpi=300)
plt.show()


## Contacts before and after of the match

We study 8 hours before of the match, 2 hours during the match and 2 hours aftewards. We normalized the number of contacts per hour and give to each curve a color depending on the city

In [ ]:
def relevant_entries(heatmap_data, target_time):
    """
    give us entries for four hours before and two hours after the target time.
    
    Parameters:
    heatmap_data (numpy array): An array of size (24, 1) representing heatmap data for 24 hours.
    target_time (int): The target hour in the range of 0 to 23.
    
    Returns:
    int: The sum of the entries for two hours before and three hours after the target time.
    """
    heatmap_data = heatmap_data.flatten()
    print(heatmap_data)
    # Calculate the indices for two hours before and three hours after the target time
    start_index = (target_time - 8) % 24
    end_index = (target_time + 4) % 24
    
    # Handle the case where the interval wraps around midnight
    if start_index < end_index:
        relevant_entries = heatmap_data[start_index:end_index+1]
    else:
        relevant_entries = np.concatenate((heatmap_data[start_index:], heatmap_data[:end_index+1]))
    print('relevant_entries',relevant_entries)
    return relevant_entries

In [ ]:
import pandas as pd
import binascii
from shapely import wkb
from pyproj import Transformer
import networkx as nx
from datetime import datetime
import numpy as np
from collections import defaultdict
slots=24


def matches_contacts(city, date):
    days_list = [date]  # Assuming we're dealing with one date at a time for simplicity
    
    for day in days_list:
        print(day)
        query = f"""
            SELECT *
            FROM (
                SELECT cnv.*, t2p.*
                FROM covid_network_sdkv6_tl7_10m cnv 
                JOIN tile2plz t2p ON cnv.tile_id = t2p.tile_id
                WHERE "day" ='{day}' 
            ) t1
            JOIN covid_plz_populate cpp ON t1.plz = cpp.id_plz5 
            WHERE city= '{city}' 
        """

        didlist = []
        pairlist = []

        df_day = pd.DataFrame(pd.read_sql_query(query, conn))
        print(df_day)
        transformer = Transformer.from_crs("epsg:3857", "epsg:4326")

        for dids, sources, in_stad, point, stime in zip(df_day.dids, df_day.sources, df_day.dist_stads, df_day.geopoint, df_day.stime):
            ngps = 1 if (in_stad == True) else 0
            bytes_object = binascii.unhexlify(point)
            geometry = wkb.loads(bytes_object)
            lon, lat = transformer.transform(geometry.x, geometry.y)
            for i, si in zip(dids, sources[1:-1].split(',')):
                for j, sj in zip(dids, sources[1:-1].split(',')):
                    if i < j and [si, sj].count('GPS') >= ngps:
                        didlist.append(i)
                        didlist.append(j)
                        pairlist.append((i, j, (lon, lat, stime)))

        print(f"Checkpoint 1 at: {datetime.now()}")
        didlist, pairlist = list(set(didlist)), list(set(pairlist))
        print(len(didlist))
        G = nx.Graph()
        G.add_nodes_from(didlist)
        print(f"Checkpoint 2 at: {datetime.now()}")
        pairlist_multiple = consolidate_list(pairlist)
        pairlist_multiple = [(node1, node2, sort_time(another_list)) for node1, node2, another_list in pairlist_multiple]
        G.add_weighted_edges_from(pairlist_multiple)
        print(len(G.edges()))
        edges = list(G.edges(data=True))

        data_dict = {}
        for edge in edges:
            node1, node2, attr = edge
            for weight in attr['weight']:
                matrix = np.zeros((1, slots), dtype=int)  # 24 hours * 6 intervals per hour
                gps_point = weight[:2]
                timestamp = weight[2]
                if timestamp.tzinfo is not None:
                    timestamp = timestamp.tz_localize(None)
                hour = timestamp.hour
                minute = timestamp.minute
                interval_index = (hour * 60 + minute) // (1* 60)

                matrix[0][interval_index] = 1
                key = (node1, node2, day)
                if key in data_dict:
                    data_dict[key]['vector'] += matrix.flatten()
                    data_dict[key]['gps_points'].append(gps_point)
                else:
                    data_dict[key] = {
                        'did1': node1,
                        'did2': node2,
                        'vector': matrix.flatten(),
                        'gps_points': [gps_point],
                        'day': day
                    }
                    
        number_of_days = 1
        # Initialize the organized data structure
        organized_data = defaultdict(lambda: np.zeros((number_of_days, slots)))  # Default to zero matrix

        # Populate the data structure
        for (did1, did2, date_str), info in data_dict.items():
            date = datetime.strptime(date_str, '%Y-%m-%d')
            index = 0

            # Ensure the index is within the valid range
            if 0 <= index < number_of_days:
                organized_data[(did1, did2)][index, :] = info['vector']
            else:
                print(f"Index {index} is out of bounds for ({did1}, {did2}) with date {date_str}")

        # List to hold the DataFrame rows
        df_rows = []

        # Iterate through the organized data
        for (did1, did2), vectors in organized_data.items():
            # Create a full vector matrix initialized with zeros
            full_vector_matrix = np.zeros((number_of_days, slots))

            # Directly assign the vectors to the corresponding dates
            full_vector_matrix[:vectors.shape[0], :vectors.shape[1]] = vectors

            # Append to the DataFrame rows
            df_rows.append({'did1': did1, 'did2': did2, 'vector': full_vector_matrix})

        # Create the DataFrame
        df = pd.DataFrame(df_rows)

        # Flatten the vector column efficiently using np.reshape
        df['flattened_vector'] = df['vector'].apply(lambda x: x.reshape(-1))

        print('vector', len(df['flattened_vector'][0]))
        
        number_of_days = len(df['flattened_vector'].iloc[0]) // slots# Assuming all vectors are the same length and correctly formatted
        print(number_of_days)
        heatmap_data = np.zeros((1, slots))
        for _, row in df.iterrows():
            matrix = np.array(row['vector']).reshape(number_of_days, slots)
            heatmap_data += matrix

        # Flip the heatmap data vertically to display 0 to 23 from bottom to top
        heatmap_data = np.flipud(heatmap_data)
        print(heatmap_data)
        

                    

        return heatmap_data
    

In [ ]:
results = []
matches = []

# Iterate through the DataFrame rows
for index, row in combined_df.iterrows():
    city = row['City']
    date = row['Date']
    
    # Get the heatmap data for the current city and date
    heatmap_data = matches_contacts(city, date)
    
    # Extract the necessary information from the row
    time = row['Time']
    match = row['Match']
    
    # Extract the target hour from the time string
    target_hour = int(time.split(':')[0])
    print(f"Processing {city} on {date} at {time} for match {match}")
    print('Current heatmap:', heatmap_data)
    print('Target hour:', target_hour)
    
    # Get relevant entries from the heatmap data
    result = relevant_entries(heatmap_data, target_hour)
    print('Result:', result)
    
    # Append the results and matches to their respective lists
    results.append(result)
    matches.append(match)

# Add the results and matches to the DataFrame
combined_df['Result'] = results
combined_df['MatchLabel'] = matches


In [ ]:
len(combined_df['Result'][0])

In [ ]:
import numpy as np

# Function to normalize a list of numbers
def normalize_list(lst):
    total = sum(lst)
    return [x / total for x in lst] if total != 0 else lst

# Apply normalization to the 'Result' column
combined_df['NormalizedResult'] = combined_df['Result'].apply(lambda x: normalize_list(x))

# Now combined_df has a new column 'NormalizedResult' with the normalized values


In [ ]:
combined_df

In [ ]:
len(combined_df['NormalizedResult'][0])

In [ ]:
import matplotlib.pyplot as plt

# Extract unique cities for color differentiation
cities = combined_df['City'].unique()
colors = plt.cm.get_cmap('tab10', len(cities))

# Create a dictionary to store colors for each city
city_colors = {city: colors(i) for i, city in enumerate(cities)}

plt.figure(figsize=(10, 6))

# Plot each row in NormalizedResult with a different color per city
for _, row in combined_df.iterrows():
    city = row['City']
    plt.plot(range(1, 14), row['NormalizedResult'], color=city_colors[city], label=city)

# Add labels and title
plt.xlabel('Entry Number')
plt.ylabel('Normalized Value')
plt.title('Normalized Result by City')

# Create a custom legend with unique cities
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys(), title="City", loc='upper right')

# Display the plot
plt.show()



In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Normalize the color map based on the time of the match
times = combined_df['Time'].apply(lambda x: int(x.split(':')[0]) + int(x.split(':')[1])/60)  # Convert time to decimal hours
norm = plt.Normalize(vmin=times.min(), vmax=times.max())
cmap = plt.cm.viridis

plt.figure(figsize=(10, 6))

# Plot each row in NormalizedResult with color based on time of the match
for _, row in combined_df.iterrows():
    time = int(row['Time'].split(':')[0]) + int(row['Time'].split(':')[1])/60
    plt.plot(range(1, 14), row['NormalizedResult'], color=cmap(norm(time)))

# Add color bar to represent time
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm)
cbar.set_label('Time of Match (hours)')

# Add labels and title
plt.xlabel('Entry Number')
plt.ylabel('Normalized Value')
plt.title('Normalized Result Colored by Time of Match')

# Display the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Filter the DataFrame to include only rows where Time is '18:00'
filtered_df = combined_df[combined_df['Time'] == '18:00']

# Extract unique cities for color differentiation
cities = filtered_df['City'].unique()
colors = plt.cm.get_cmap('tab10', len(cities))

# Create a dictionary to store colors for each city
city_colors = {city: colors(i) for i, city in enumerate(cities)}

plt.figure(figsize=(10, 6))

# Plot each row in NormalizedResult with a different color per city
for _, row in filtered_df.iterrows():
    city = row['City']
    plt.plot(range(1, 14), row['NormalizedResult'], color=city_colors[city], label=city)

# Add labels and title
plt.xlabel('Entry Number')
plt.ylabel('Normalized Value')
plt.title('Normalized Result by City for Time 18:00')

# Create a custom legend with unique cities
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys(), title="City", loc='upper right')

# Display the plot
plt.show()



In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assume the 'Population' column exists in the combined_df with the population size for each city

# Classify cities based on population size
small_cities = combined_df[combined_df['Population'] < 600000]['City'].unique()
medium_cities = combined_df[(combined_df['Population'] >= 600_000) & (combined_df['Population'] < 1_000_000)]['City'].unique()
large_cities = combined_df[combined_df['Population'] >= 1_000_000]['City'].unique()

# Sum the 'NormalizedResult' vectors for small, medium, and large cities
small_group_sum = combined_df[combined_df['City'].isin(small_cities)].groupby('City')['NormalizedResult'].apply(lambda x: np.sum(np.array(x.tolist()), axis=0)).sum()
medium_group_sum = combined_df[combined_df['City'].isin(medium_cities)].groupby('City')['NormalizedResult'].apply(lambda x: np.sum(np.array(x.tolist()), axis=0)).sum()
large_group_sum = combined_df[combined_df['City'].isin(large_cities)].groupby('City')['NormalizedResult'].apply(lambda x: np.sum(np.array(x.tolist()), axis=0)).sum()

plt.figure(figsize=(10, 6))

# Add grey shaded area from 4 to 6 on the x-axis
plt.axvspan(8, 10, color='grey', alpha=0.3)

# Plot the summed vectors for small, medium, and large population groups
plt.plot(range(1, 14), small_group_sum, label='Small Population (< 1 million)', color='blue')
plt.plot(range(1, 14), medium_group_sum, label='Medium Population (600k - 1 million)', color='green')
plt.plot(range(1, 14), large_group_sum, label='Large Population (> 1 million)', color='red')

# Add labels and title
plt.xlabel('hours')
plt.ylabel('Summed Normalized Value')
plt.title('Summed Normalized Results by Population Size')
plt.legend(title="Population Group", loc='upper right')

# Display the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Assuming you have a DataFrame called `combined_df` with a 'NormalizedResult' column
# which contains lists of 9 values (one per entry)

# Extract all entries into separate lists
entries = np.array(combined_df['NormalizedResult'].tolist()).T

# Calculate means and standard deviations for each entry
means = np.mean(entries, axis=1)
std_devs = np.std(entries, axis=1)

# Create the plot
plt.figure(figsize=(10, 6))
plt.axvspan(8, 10, color='grey', alpha=0.3)

# Create boxplots for each entry
sns.boxplot(data=entries.T, palette="pastel")

# Plot the means and connect them with a line
plt.plot(range(13), means, marker='o', color='black', label='Mean Value')

# Add error bars for standard deviations
plt.errorbar(range(13), means, yerr=std_devs, fmt='o', color='black', capsize=5)
plt.xticks(ticks=range(0, 13), labels=range(-8, 5))
# Add labels and title
plt.xlabel('Entry Number')
plt.ylabel('Value')
plt.title('Distribution of Each Entry with Mean and Standard Deviation')
plt.legend()

# Display the plot
plt.show()


In [ ]:
# Get the unique city names with population under 1.5 million
small_cities = combined_df[combined_df['Population'] < 600000]['City'].unique()

# Filter the DataFrame for those small cities
small_cities_df = combined_df[combined_df['City'].isin(small_cities)]


In [ ]:
small_cities_df

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Extract all entries into separate lists for small cities
entries = np.array(small_cities_df['NormalizedResult'].tolist()).T

# Calculate means and standard deviations for each entry
means = np.mean(entries, axis=1)
std_devs = np.std(entries, axis=1)

# Create the plot
plt.figure(figsize=(10, 6))

# Add grey shaded area from 3 to 5 on the x-axis
plt.axvspan(7, 9, color='grey', alpha=0.3)

# Create boxplots for each entry
sns.boxplot(data=entries.T, palette="pastel")

# Plot the means and connect them with a line
plt.plot(range(13), means, marker='o', color='black', label='Mean Value')

# Add error bars for standard deviations
plt.errorbar(range(13), means, yerr=std_devs, fmt='o', color='black', capsize=5)
# Set x-ticks from -3 to 5
plt.xticks(ticks=range(13), labels=range(-7, 6))

# Add labels and title
plt.xlabel('hours')
plt.ylabel(' normalized number of contacts')
plt.title('distribution of the normalized number of contact per hour with for small cities (under 600k)')
plt.legend()

# Display the plot
plt.show()


In [ ]:
# Get the unique city names with population between 600,000 and 1 million
medium_cities = combined_df[(combined_df['Population'] > 600_000) & (combined_df['Population'] < 1_000_000)]['City'].unique()

# Filter the DataFrame for those medium cities
medium_cities_df = combined_df[combined_df['City'].isin(medium_cities)]



In [ ]:
medium_cities_df

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Assuming you have already defined `large_cities_df` by filtering `combined_df`

# Extract all entries into separate lists for large cities
entries = np.array(medium_cities_df['NormalizedResult'].tolist()).T

# Calculate means and standard deviations for each entry
means = np.mean(entries, axis=1)
std_devs = np.std(entries, axis=1)

# Create the plot
plt.figure(figsize=(10, 6))

# Add grey shaded area from 3 to 5 on the x-axis
plt.axvspan(7, 9, color='grey', alpha=0.3)

# Create boxplots for each entry
sns.boxplot(data=entries.T, palette="pastel")

# Plot the means and connect them with a line
plt.plot(range(13), means, marker='o', color='black', label='Mean Value')

# Add error bars for standard deviations
plt.errorbar(range(13), means, yerr=std_devs, fmt='o', color='black', capsize=5)

# Set x-ticks from -3 to 5
plt.xticks(ticks=range(13), labels=range(-7, 6))

# Add labels and title
plt.xlabel('Entry Number (-3 to 5)')
plt.ylabel('Value')
plt.title('Distribution of Each Entry with Mean and Standard Deviation for Large Cities')
plt.legend()

# Display the plot
plt.show()


In [ ]:
# Get the unique city names with population under 1.5 million
large_cities = combined_df[combined_df['Population'] > 1000000]['City'].unique()

# Filter the DataFrame for those small cities
large_cities_df = combined_df[combined_df['City'].isin(large_cities)]


In [ ]:
large_cities_df

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Assuming you have already defined `large_cities_df` by filtering `combined_df`

# Extract all entries into separate lists for large cities
entries = np.array(large_cities_df['NormalizedResult'].tolist()).T

# Calculate means and standard deviations for each entry
means = np.mean(entries, axis=1)
std_devs = np.std(entries, axis=1)

# Create the plot
plt.figure(figsize=(10, 6))

# Add grey shaded area from 3 to 5 on the x-axis
plt.axvspan(7, 9, color='grey', alpha=0.3)

# Create boxplots for each entry
sns.boxplot(data=entries.T, palette="pastel")

# Plot the means and connect them with a line
plt.plot(range(13), means, marker='o', color='black', label='Mean Value')

# Add error bars for standard deviations
plt.errorbar(range(13), means, yerr=std_devs, fmt='o', color='black', capsize=5)

# Set x-ticks from -3 to 5
plt.xticks(ticks=range(13), labels=range(-7, 6))

# Add labels and title
plt.xlabel('Entry Number (-3 to 5)')
plt.ylabel('Value')
plt.title('Distribution of Each Entry with Mean and Standard Deviation for Large Cities')
plt.legend()

# Display the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Assume the combined_df DataFrame is already loaded and contains 'City', 'Population', and 'NormalizedResult'

# Expand the 'NormalizedResult' column into separate columns for each entry
expanded_df = pd.DataFrame(combined_df['NormalizedResult'].tolist(), index=combined_df.index)
expanded_df.columns = [f'Entry {i+1}' for i in range(expanded_df.shape[1])]

# Add back the 'City' and 'Population' columns
expanded_df['City'] = combined_df['City']
expanded_df['Population'] = combined_df['Population']

# Melt the DataFrame to get a long-form DataFrame suitable for seaborn
melted_df = pd.melt(expanded_df, id_vars=['City', 'Population'], var_name='Entry', value_name='Normalized Value')

# Map each city's population to point size
melted_df['Size'] = melted_df['Population'] / 100000  # Adjust scaling factor as needed

# Create the plot
plt.figure(figsize=(12, 8))

# Plot using Seaborn scatterplot
sns.scatterplot(data=melted_df, x='Entry', y='Normalized Value', 
                hue='City', size='Size', sizes=(20, 200), alpha=0.7, palette='tab10')


# Set x-ticks from -3 to 5
plt.xticks(ticks=range(13), labels=range(-7, 6))
plt.axvspan(7, 9, color='grey', alpha=0.3)

# Add labels and title
plt.xlabel('Entry')
plt.ylabel('Normalized Value')
plt.title('Scatter Plot of Normalized Results by City with Population Size')
plt.legend(title="City", loc='upper right', bbox_to_anchor=(1.25, 1))

# Display the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Assume the combined_df DataFrame is already loaded and contains 'City', 'Population', 'Time', and 'NormalizedResult'

# Expand the 'NormalizedResult' column into separate columns for each entry
expanded_df = pd.DataFrame(combined_df['NormalizedResult'].tolist(), index=combined_df.index)
expanded_df.columns = [f'Entry {i+1}' for i in range(expanded_df.shape[1])]

# Add back the 'City', 'Population', and 'Time' columns
expanded_df['City'] = combined_df['City']
expanded_df['Population'] = combined_df['Population']
expanded_df['Time'] = combined_df['Time']

# Melt the DataFrame to get a long-form DataFrame suitable for seaborn
melted_df = pd.melt(expanded_df, id_vars=['City', 'Population', 'Time'], 
                    var_name='Entry', value_name='Normalized Value')

# Map each city's population to point size
melted_df['Size'] = melted_df['Population'] / 100000  # Adjust scaling factor as needed

# Filter to include only the times of interest
filtered_df = melted_df[melted_df['Time'].isin(['15:00', '18:00', '21:00'])]

# Create the plot
plt.figure(figsize=(12, 8))

# Plot using Seaborn scatterplot
sns.scatterplot(data=filtered_df, x='Entry', y='Normalized Value', 
                hue='City', size='Size', sizes=(20, 200), alpha=0.7, palette='tab10')

# Calculate and plot the mean normalized values for each entry by time
mean_values = filtered_df.groupby(['Entry', 'Time'])['Normalized Value'].mean().unstack()

# Plot lines connecting mean points for each time
plt.plot(range(13), mean_values['15:00'], marker='o', color='blue', linestyle='-', label='15:00 Mean')
plt.plot(range(13), mean_values['18:00'], marker='o', color='green', linestyle='-', label='18:00 Mean')
plt.plot(range(13), mean_values['21:00'], marker='o', color='red', linestyle='-', label='21:00 Mean')


# Set x-ticks from -3 to 5
plt.xticks(ticks=range(13), labels=range(-7, 6))
plt.axvspan(7, 9, color='grey', alpha=0.3)
plt.xlabel('hours')
plt.ylabel('Normalized Value')
plt.title('Scatter Plot of Normalized Results by City with Time-Specific Means')
plt.legend(title="City and Time", loc='upper right', bbox_to_anchor=(1.25, 1))

# Display the plot
plt.show()


## Fig_6 

In [ ]:
import ast
combined_df=pd.read_csv('data/fig_6/number_of_contacts_per_day_per_match_tl7_10m_fig_6.csv')
combined_df["NormalizedResult"] = combined_df["NormalizedResult"].apply(ast.literal_eval)
combined_df
combined_df['MatchType'] = np.where(
    combined_df['Match'].str.contains("Germany"), 
    "Germany", 
    "Other"
)
combined_df['HourOffsets'] = [list(range(-7, 6))] * len(combined_df)

# Explode both the NormalizedResult and HourOffsets columns 
# so that each (hour offset, normalized value) pair becomes its own row
df_long = combined_df.explode(['NormalizedResult', 'HourOffsets']).reset_index(drop=True)

In [ ]:
plt.figure(figsize=(10, 6))

sns.boxplot(
    data=df_long,
    x='HourOffsets',
    y='NormalizedResult',
    hue='MatchType',
    palette={'Germany': 'blue', 'Other': 'orange'}
)

# Optionally shade the match window, e.g. from hour 0 to hour 2
plt.axvspan(6.6, 9.4, color='grey', alpha=0.3)

# Clean up axes and legend
plt.xticks(ticks=range(0, 13), labels=range(-7, 6))
plt.xlabel('time around the match in hours')
plt.ylabel('normalized number of contacts')
plt.legend(title='match type')
plt.tight_layout()
plt.savefig('plots/fig_6/fig_6_tl7_10m.png', format="png", dpi=300)  # 'png' should be a string

plt.show()


In [ ]:
# Assuming `combined_df` is the main dataframe
# Filter for rows where the match involves Germany
germany_matches_df = combined_df[combined_df['Match'].str.contains("Germany")].reset_index(drop=True)

# Extract the time information and convert it to a numerical value for coloring
germany_matches_df['Hour'] = pd.to_datetime(germany_matches_df['Time']).dt.hour
# Extract all entries into separate lists for Germany matches
entries = np.array(germany_matches_df['NormalizedResult'].tolist()).T
# Define x_ticks from -9 to 6
x_ticks = list(range(-7, 6))

# Create the plot
plt.figure(figsize=(10, 6))

# Plot a line for each German match with a unique color
num_matches = len(germany_matches_df)
colors = plt.cm.viridis(np.linspace(0, 1, num_matches))

for index, row in germany_matches_df.iterrows():
    plt.plot(range(len(row['NormalizedResult'])), row['NormalizedResult'], marker='o', label=row['Match'], color=colors[index])

# Set the x-ticks
entries = np.array(germany_matches_df['NormalizedResult'].tolist())

plt.xticks(ticks=range(len(entries[0])), labels=x_ticks[:len(entries[0])])
plt.axvspan(7, 9, color='grey', alpha=0.3)

# Add labels and title
plt.xlabel('Entry Number')
plt.ylabel('Normalized Value')
#plt.title('Normalized Results of German Matches with Unique Colors for Each Match')
plt.legend()

# Display the plot
plt.show()

## Heatmaps stadiums per city


In [ ]:
slots = 24
start_date = datetime.strptime('2024-05-01', '%Y-%m-%d')
end_date = datetime.strptime('2024-07-30', '%Y-%m-%d')
#city='Gelsenkirchen'


# Event details from combined_df
event_details = combined_df[['City', 'Date', 'Time', 'Match']]

# Convert Date column to datetime
event_details['Date'] = pd.to_datetime(event_details['Date'])

# Extract event times and dates
event_times = {
    (row['City'], datetime.strptime(row['Date'].strftime('%Y-%m-%d') + ' ' + row['Time'], '%Y-%m-%d %H:%M')): row['Match']
    for index, row in event_details.iterrows()
    if start_date <= row['Date'] <= end_date
}


# Generate the list of days
delta = end_date - start_date
days_list = [(start_date + timedelta(days=i)).strftime('%Y-%m-%d') for i in range(delta.days + 1)]
print(days_list)

df_total = pd.DataFrame()
data_dict = {}
for day in days_list:
    query = f"""
    WITH filtered_cnv AS (
        SELECT cnv.*, t2p.*
        FROM covid_network_sdkv6_tl5_10m cnv
        JOIN tile2plz t2p ON cnv.tile_id = t2p.tile_id
        WHERE "day" = '{day}'
    ), city_filtered AS (
        SELECT filtered_cnv.*
        FROM filtered_cnv
        JOIN covid_plz_populate cpp ON filtered_cnv.plz = cpp.id_plz5
        WHERE cpp.city = '{city}'
    )
    SELECT cf.*, tbo.*
    FROM city_filtered cf
    LEFT JOIN planet_osm_polygon tbo ON ST_Intersects(cf.geopoint, tbo.way)
    WHERE tbo.leisure = 'stadium';
    """
    didlist=[]
    pairlist=[]

    df_day= pd.DataFrame(pd.read_sql_query(query, conn))
    transformer = Transformer.from_crs("epsg:3857", "epsg:4326")

    # iterate over contact events
    for dids, sources, in_stad, point, time in zip(df_day.dids, df_day.sources, df_day.dist_stads, df_day.geopoint, df_day.stime):
        ngps = 1 if (in_stad==True) else 0
        for i, si in zip(dids, sources[1:-1].split(',')):
            for j, sj in zip(dids, sources[1:-1].split(',')):
                if i < j and [si, sj].count('GPS') >= ngps:
                    # collect dids in did list
                    didlist.append(i)
                    didlist.append(j)
                    bytes_object = binascii.unhexlify(point)
                    geometry = wkb.loads(bytes_object)
                    # Transformer from EPSG:3857 to EPSG:4326

                    # Assuming that `geometry` is a Point
                    lon, lat = transformer.transform(geometry.x, geometry.y)
                    # collect pairs of dids in pair list
                    pairlist.append((i, j, (lon, lat, time)))

    # get unique dids and did pairs
    print(f"Checkpoint 1 at: {datetime.now()}")
    didlist, pairlist = list(set(didlist)), list(set(pairlist))
    print(len(didlist))

    G = nx.Graph()
    G.add_nodes_from(didlist)
    print(f"Checkpoint 2 at: {datetime.now()}")

    pairlist_multiple = consolidate_list(pairlist)
    pairlist_multiple = [(node1, node2, sort_time(another_list)) for node1, node2, another_list in pairlist_multiple]
    G.add_weighted_edges_from(pairlist_multiple)
    print(len(G.edges()))
    print(len(G.nodes()))
    edges = list(G.edges(data=True))

    for edge in edges:
        node1, node2, attr = edge
        for weight in attr['weight']:
            matrix = np.zeros((1, slots), dtype=int)
            gps_point = weight[:2]
            timestamp = weight[2]
            if timestamp.tzinfo is not None:
                timestamp = timestamp.tz_localize(None)
            hour = timestamp.hour
            minute = timestamp.minute
            interval_index = (hour * 60 + minute) // 60
            matrix[0][interval_index] = 1
            key = (node1, node2, day)
            if key in data_dict:
                data_dict[key]['vector'] += matrix.flatten()
                data_dict[key]['gps_points'].append(gps_point)
            else:
                data_dict[key] = {
                    'did1': node1,
                    'did2': node2,
                    'vector': matrix.flatten(),
                    'gps_points': [gps_point],
                    'day': day
                }
    


In [ ]:
date_range = [start_date + timedelta(days=x) for x in range((end_date - start_date).days + 1)]
number_of_days = len(date_range)
organized_data = defaultdict(lambda: np.zeros((number_of_days, slots)))


for (did1, did2, date_str), info in data_dict.items():
    date = datetime.strptime(date_str, '%Y-%m-%d')
    index = (date - start_date).days
    if 0 <= index < number_of_days:
        organized_data[(did1, did2)][index, :] = info['vector']
    else:
        print(f"Index {index} is out of bounds for ({did1}, {did2}) with date {date_str}")

df_rows = []

for (did1, did2), vectors in organized_data.items():
    # Create a full vector matrix initialized with zeros
    full_vector_matrix = np.zeros((number_of_days, slots))

    # Directly assign the vectors to the corresponding dates
    full_vector_matrix[:vectors.shape[0], :vectors.shape[1]] = vectors
    # Sum the slots per day
    daily_sums = full_vector_matrix.sum(axis=1)


    # Append to the DataFrame rows
    df_rows.append({'pair': f'{did1}-{did2}', 'vector': full_vector_matrix, 'daily_sums': daily_sums})


In [ ]:
df = pd.DataFrame(df_rows)
df['flattened_vector'] = df['vector'].apply(lambda x: x.reshape(-1))

number_of_days = len(df['flattened_vector'].iloc[0]) // slots
heatmap_data = np.zeros((number_of_days, slots))
for _, row in df.iterrows():
    matrix = np.array(row['vector']).reshape(number_of_days, slots)
    heatmap_data += matrix
heatmap_data_df = pd.DataFrame(heatmap_data)




In [ ]:
#city='Leipzig'
#heatmap_data_df=pd.read_csv(f'heatmap_data_stadiums_Leipzig.csv')
#heatmap_data = heatmap_data_df.to_numpy()

In [ ]:
heatmap_data_df

In [ ]:
#heatmap_data_df.to_csv(f'heatmap_data_stadiums_{city}.csv', index=False)
# Flip the data for correct orientation
heatmap_data = np.flipud(heatmap_data.T)

In [ ]:
import matplotlib.colors as mcolors

sns.set_context("notebook")
sns.set_style("white")

# Assuming heatmap_data, start_date, number_of_days, slots, city, and event_times are already defined

vmax_auto = np.max(heatmap_data)
print(vmax_auto)
fig, ax = plt.subplots(figsize=(40, 15))

# Create the heatmap without the color bar
cax = sns.heatmap(heatmap_data, cmap='YlGnBu', vmin=0, vmax=150, ax=ax, linewidths=.5, cbar=False)

# Generate all dates within the range
dates = [start_date + timedelta(days=i) for i in range(number_of_days)]

# Identify all Sundays
sundays = [date for date in dates if date.weekday() == 6]

# Select every other Sunday
sundays_every_two_weeks = sundays[::2]

# Positions of the selected Sundays in the dates list
sunday_positions = [dates.index(sunday) for sunday in sundays_every_two_weeks]

# Set x-ticks at the positions of the selected Sundays
ax.set_xticks(sunday_positions)

# Set x-tick labels as the formatted dates
xtick_labels = [sunday.strftime('%m-%d') for sunday in sundays_every_two_weeks]
ax.set_xticklabels(xtick_labels, ha='right', fontsize=60)

# Set y-ticks every 4 hours
ax.set_yticks([i + 3.5 for i in range(0, slots, 4)])  # Ticks at 0, 4, 8, 12, 16, 20

# Set y-tick labels corresponding to every 4 hours
ax.set_yticklabels(reversed(range(0, slots, 4)), fontsize=60, rotation=0)

# Move the y-axis label slightly to the right
ax.set_ylabel("hour of the day", fontsize=70, labelpad=30)  # Adjust 'labelpad' as needed

ax.set_title(f"stadiums in {city}", fontsize=80)

# Define a threshold for deciding if the text color should be orange
threshold=10# Adjust this value based on your data

# Plot events
for (event_city, event_time), event_name in event_times.items():
    if event_city == city:
        event_day_index = (event_time.date() - start_date.date()).days
        event_hour_index = slots - 1 - event_time.hour  # Adjusted for 24 slots

        match_parts = event_name.split(" ")
        team1 = match_parts[0][:3].upper()
        score1 = match_parts[1]
        score2 = match_parts[3]
        team2 = match_parts[4][:3].upper()

        team_abbreviation = f"{team1}-{team2}"
        
        # Determine if the value in heatmap_data is below the threshold
        cell_value = heatmap_data[event_hour_index, event_day_index]
        if cell_value < threshold:
            text_color = 'black'
            font_weight = 'normal'
        else:
            text_color = 'red'
            font_weight = 'bold'
        
        ax.text(event_day_index + 0.5, event_hour_index + 0.5, team_abbreviation,
                ha='center', va='center', color=text_color, fontsize=30, rotation=90)

plt.tight_layout()
plt.show()

In [ ]:
#FF450

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors

# Define the maximum and minimum values
max_value = 150
min_value = 0

# Use the 'YlGnBu' colormap
cmap = plt.get_cmap('YlGnBu')

# Create a gradient array for the bar
gradient = np.linspace(0, 1, 256).reshape(1, -1)

# Plot the horizontal bar with the gradient
fig, ax = plt.subplots(figsize=(10, 1))  # Adjust size as needed

# Display the gradient
ax.imshow(gradient, aspect='auto', cmap=cmap, extent=[min_value, max_value, 0, 1])

# Add "Min" and "Max" labels
ax.text(min_value, 0.5, 'Min', va='center', ha='left', fontsize=20, color='black')
ax.text(max_value, 0.5, 'Max', va='center', ha='right', fontsize=20, color='#FF4500')

# Hide y-axis labels and ticks since we don't need them
ax.yaxis.set_visible(False)

# Hide x-axis ticks but keep the line
ax.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)

# Remove the frame (optional)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

plt.show()



## POI

In [ ]:
# Updated start_date
start_date = datetime.strptime('2024-05-01', '%Y-%m-%d')
end_date = datetime.strptime('2024-07-30', '%Y-%m-%d')



In [ ]:
# Event details from combined_df
event_details = combined_df[['City', 'Date', 'Time', 'Match']]

# Convert Date column to datetime
event_details['Date'] = pd.to_datetime(event_details['Date'])

# Extract event times and dates
event_times = {
    (row['City'], datetime.strptime(row['Date'].strftime('%Y-%m-%d') + ' ' + row['Time'], '%Y-%m-%d %H:%M')): row['Match']
    for index, row in event_details.iterrows()
    if start_date <= row['Date'] <= end_date
}

In [ ]:
german_match_dates = {
    pd.to_datetime("2024-06-14"): "GER-SCO",
    pd.to_datetime("2024-06-19"): "GER-HUN",
    pd.to_datetime("2024-06-23"): "GER-SUI",
    pd.to_datetime("2024-06-29"): "GER-DEN",
    pd.to_datetime("2024-07-05"): "GER-SPA"
}
 
turkey_match_dates = {
    pd.to_datetime("2024-06-18"): "TUR-GEO",
    pd.to_datetime("2024-06-22"): "TUR-POR",
    pd.to_datetime("2024-06-26"): "TUR-CZE",
    pd.to_datetime("2024-07-02"): "TUR-AUS",
    pd.to_datetime("2024-07-06"): "TUR-NED"
}
 
england_match_dates = {
    pd.to_datetime("2024-06-16"): "ENG-SER",
    pd.to_datetime("2024-06-20"): "ENG-DEN",
    pd.to_datetime("2024-06-25"): "ENG-SLO",
    pd.to_datetime("2024-06-30"): "ENG-SVK",
    pd.to_datetime("2024-07-06"): "ENG-SUI",
    pd.to_datetime("2024-07-10"): "ENG-NED",
    pd.to_datetime("2024-07-14"): "ENG-SPA"
}

In [ ]:
df=pd.read_csv('heatmap_POI_germany_euro_2024.csv')

In [ ]:
df

In [ ]:
# Flip the heatmap data vertically to display 0 to 23 from bottom to top
heatmap_data = df.to_numpy()
heatmap_data= np.flipud(heatmap_data.T)
 
sns.set_context("notebook")
sns.set_style("white")

# Assuming heatmap_data, start_date, number_of_days, slots, city, and event_times are already defined

vmax_auto = np.max(heatmap_data)
print(vmax_auto)
fig, ax = plt.subplots(figsize=(40, 15))

# Create the heatmap without the color bar
cax = sns.heatmap(heatmap_data, cmap='YlGnBu', vmin=0, vmax=100
                  , ax=ax, linewidths=.5, cbar=False)

# Generate all dates within the range
dates = [start_date + timedelta(days=i) for i in range(number_of_days)]

# Identify all Sundays
sundays = [date for date in dates if date.weekday() == 6]

# Select every other Sunday
sundays_every_two_weeks = sundays[::2]

# Positions of the selected Sundays in the dates list
sunday_positions = [dates.index(sunday) for sunday in sundays_every_two_weeks]

# Set x-ticks at the positions of the selected Sundays
ax.set_xticks(sunday_positions)

# Set x-tick labels as the formatted dates
xtick_labels = [sunday.strftime('%m-%d') for sunday in sundays_every_two_weeks]
ax.set_xticklabels(xtick_labels, ha='right', fontsize=60)

# Set y-ticks every 4 hours
ax.set_yticks([i + 3.5 for i in range(0, slots, 4)])  # Ticks at 0, 4, 8, 12, 16, 20

# Set y-tick labels corresponding to every 4 hours
ax.set_yticklabels(reversed(range(0, slots, 4)), fontsize=60, rotation=0)

# Move the y-axis label slightly to the right
ax.set_ylabel("hour of the day", fontsize=80, labelpad=30)  # Adjust 'labelpad' as needed

ax.set_title('POI', fontsize=80)

# Define a threshold for deciding if the text color should be orange
threshold = 40  # Adjust this value based on your data


# Plot events
# Define the list of target team abbreviations
target_teams = {"GER"}

for (event_city, event_time), event_name in event_times.items():
    
    event_day_index = (event_time.date() - start_date.date()).days
    event_hour_index = slots - 1 - event_time.hour  # Adjusted for 24 slots

    match_parts = event_name.split(" ")
    team1 = match_parts[0][:3].upper()
    score1 = match_parts[1]
    score2 = match_parts[3]
    team2 = match_parts[4][:3].upper()

    team_abbreviation = f"{team1}-{team2}"

    # Check if either team1 or team2 is in the target teams
    if team1 in target_teams or team2 in target_teams:
        # Determine if the value in heatmap_data is below the threshold
        cell_value = heatmap_data[event_hour_index, event_day_index]
        if cell_value < threshold:
            text_color = 'black'
            font_weight = 'normal'
        else:
            text_color = '#FF4500'
            font_weight = 'bold'

        ax.text(event_day_index + 0.5, event_hour_index + 0.5, team_abbreviation,
                ha='center', va='center', color=text_color, fontsize=30, rotation=90)

plt.tight_layout()
plt.show()
 
# Calculate projections on the x and y axes
x_projection = np.sum(heatmap_data, axis=0)
y_projection = np.sum(heatmap_data, axis=1)

# Calculate the average contacts per day type
day_of_week_contacts = {i: [] for i in range(7)}  # Dictionary to hold lists of contacts for each day of the week

for i, date in enumerate(dates):
    day_of_week_contacts[date.weekday()].append(x_projection[i])

# Calculate the average contacts for each day of the week
avg_day_of_week_contacts = {day: np.mean(contacts) for day, contacts in day_of_week_contacts.items()}

# Normalize the x_projection by the average contacts for each day type
normalized_x_projection = np.array([x_projection[i] / avg_day_of_week_contacts[date.weekday()] for i, date in enumerate(dates)])

# Plot the normalized x-axis projection histogram
plt.figure(figsize=(20, 15))
colors = [
    'purple' if date in german_match_dates else
    'skyblue' if date in turkey_match_dates else
    'green' if date in england_match_dates else
    'grey'
    for date in dates
]  # Highlight match dates

bars = plt.bar(range(len(dates)), normalized_x_projection, color=colors)  # Ensure the range matches the number of days

# Generate all dates within the range
dates = [start_date + timedelta(days=i) for i in range(number_of_days)]

# Identify all Sundays
sundays = [date for date in dates if date.weekday() == 6]

# Select every other Sunday
sundays_every_two_weeks = sundays[::2]

# Positions of the selected Sundays in the dates list
sunday_positions = [dates.index(sunday) for sunday in sundays_every_two_weeks]

# Set x-ticks at the positions of the selected Sundays
plt.xticks(sunday_positions)
plt.yticks([])

# Set x-tick labels as the formatted dates
xtick_labels = [sunday.strftime('%m-%d') for sunday in sundays_every_two_weeks]
plt.gca().set_xticklabels(xtick_labels, ha='right', fontsize=25)
plt.axhline(y=1, color='red', linestyle='--', linewidth=2)
# Add labels to the bars for match dates
for i, bar in enumerate(bars):
    date = dates[i]
    match_label = None
    if date in german_match_dates:
        match_label = german_match_dates[date]
    elif date in turkey_match_dates:
        match_label = turkey_match_dates[date]
    elif date in england_match_dates:
        match_label = england_match_dates[date]
    if match_label:
        plt.text(
            bar.get_x() + bar.get_width() / 2,
            bar.get_height(),
            match_label,
            ha='center',
            va='bottom',
            color='black',
            fontsize=15,
            rotation='vertical'
        )

plt.ylabel("number of contacts", fontsize=30)
plt.tight_layout()
plt.show()
# Plot the normalized x-axis projection histogram
# Plot the normalized x-axis projection histogram
plt.figure(figsize=(25, 15))
colors = [
    'purple' if date in german_match_dates else
    'skyblue' if date in turkey_match_dates else
    'green' if date in england_match_dates else
    'grey'
    for date in dates
]  # Highlight match dates

bars = plt.bar(range(len(dates)), x_projection, color=colors)

# Identify all Sundays
sundays = [date for date in dates if date.weekday() == 6]

# Select every other Sunday
sundays_every_two_weeks = sundays[::2]

# Positions of the selected Sundays in the dates list
sunday_positions = [dates.index(sunday) for sunday in sundays_every_two_weeks]

# Set x-ticks at the positions of the selected Sundays
plt.xticks(sunday_positions)

# Set x-tick labels as the formatted dates for every two Sundays
xtick_labels = [sunday.strftime('%m-%d') for sunday in sundays_every_two_weeks]
ax = plt.gca()  # Get current axis
ax.set_xticklabels(xtick_labels, ha='right', fontsize=30)

# Add labels to the bars for match dates
for i, bar in enumerate(bars):
    date = dates[i]
    match_label = None
    if date in german_match_dates:
        match_label = german_match_dates[date]
    elif date in turkey_match_dates:
        match_label = turkey_match_dates[date]
    elif date in england_match_dates:
        match_label = england_match_dates[date]
    if match_label:
        plt.text(
            bar.get_x() + bar.get_width() / 2,
            bar.get_height(),
            match_label,
            ha='center',
            va='bottom',
            color='black',
            fontsize=12,
            rotation='vertical'
        )

plt.ylabel("number of contacts", fontsize=60)
plt.yticks([])

#plt.title("Sum of Contacts Per Day", fontsize=35)
plt.tight_layout()

plt.show()
 

In [ ]:

from scipy.stats import norm
from datetime import timedelta

# Example match dates
german_match_dates = {
    pd.to_datetime("2024-06-14"): "GER-SCO",
    pd.to_datetime("2024-06-19"): "GER-HUN",
    pd.to_datetime("2024-06-23"): "GER-SUI",
    pd.to_datetime("2024-06-29"): "GER-DEN",
    pd.to_datetime("2024-07-05"): "GER-SPA"
}

# Assuming dates, heatmap_data, and x_projection are already defined.

# Group contact counts by day of the week
day_of_week_contacts = {i: [] for i in range(7)}

for i, date in enumerate(dates):
    day = date.weekday()
    day_of_week_contacts[day].append(x_projection[i])

# Compute custom statistics for each day in the range, comparing to the distribution for the same weekday
custom_stats = []
p_values = []
for i, date in enumerate(dates):
    day = date.weekday()
    contact_today = x_projection[i]
    all_other_contacts = np.array([contacts for j, contacts in enumerate(day_of_week_contacts[day]) if j != i])

    if len(all_other_contacts) > 1:
        mean_contacts = np.mean(all_other_contacts)
        std_contacts = np.std(all_other_contacts, ddof=1)
        
        if std_contacts > 0:
            custom_stat = (contact_today - mean_contacts) / std_contacts
            p_value = norm.sf(np.abs(custom_stat)) * 2  # Two-tailed test equivalent
        else:
            custom_stat = np.nan
            p_value = np.nan
        
        custom_stats.append(custom_stat)
        p_values.append(p_value)
    else:
        custom_stats.append(np.nan)
        p_values.append(np.nan)

# Plotting days vs. p-values
plt.figure(figsize=(20, 10))

# Color points based on whether the day has a match in Germany
colors = ['purple' if date in german_match_dates else 'blue' for date in dates]

plt.scatter(dates, p_values, color=colors, s=100)  # s=100 for larger dots

# Add a horizontal line at y=0.05 to indicate the significance threshold
plt.axhline(y=0.05, color='green', linestyle='--', linewidth=2, label='Significance Level (0.05)')

# Labeling the plot
plt.xticks(rotation=45)
plt.xlabel('Date', fontsize=14)
plt.ylabel('P-value', fontsize=14)
plt.title('P-values for Daily Contact Changes Compared to the Same Day of the Week', fontsize=16)

# Highlight points for matches in Germany
for i, date in enumerate(dates):
    if date in german_match_dates:
        plt.text(date, p_values[i], german_match_dates[date], fontsize=12, ha='center', color='purple')

plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

# Plotting days vs. custom statistics
plt.figure(figsize=(20, 10))

plt.scatter(dates, custom_stats, color=colors, s=100)  # s=100 for larger dots

# Labeling the plot
plt.xticks(rotation=45)
plt.xlabel('Date', fontsize=14)
plt.ylabel('Custom Statistic', fontsize=14)
plt.title('Custom Statistics for Daily Contact Changes Compared to the Same Day of the Week', fontsize=16)

# Highlight points for matches in Germany
for i, date in enumerate(dates):
    if date in german_match_dates:
        plt.text(date, custom_stats[i], german_match_dates[date], fontsize=12, ha='center', color='purple')

plt.grid(True)
plt.tight_layout()
plt.show()